In [1]:
dbutils.library.installPyPI("mlflow")
dbutils.library.restartPython()
import mlflow

Tue Sep 8 17:51:14 2020 py4j imported

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
# reading parquet files from the curated data
dirPath = '/mnt/root/COVID19_TWEETS/v3/REFINED/WITH_SENTIMENT/'

In [4]:
# read the data in batch
tweets = spark.read.parquet(dirPath)

In [5]:
# just to get a feel of how much data we are dealing with:
tweets.where(tweets['user_location'] =='US').count()

Out[3]: 47

In [6]:
tweets.where(tweets["text"]=='').count()

Out[4]: 0

In [7]:
tweets.select("user_location").distinct().count()

Out[5]: 31503

In [8]:
tweets = tweets.where(tweets["text"]!='')

In [9]:
tweets.where(tweets["text"].isNull()).count()

Out[7]: 0

In [10]:
display(tweets.limit(10))

#Leveraging Spark ML:
Data wrangling, feature exctraction and mormalization:

Tuning with pyspark.ml in databricks environment, metrics and models are automatically logged to MLflow.

In [12]:
# arrange the data to fit to our needs:
from pyspark.sql.types import DoubleType
# drop null values
tweets = tweets.dropna(subset=["positive_sentiment","negative_sentiment","is_retweet","user_location","hashtags"])
#cast values
tweets = tweets.withColumn("negative_sentiment",tweets['negative_sentiment'].cast(DoubleType()))
tweets = tweets.withColumn("positive_sentiment",tweets['positive_sentiment'].cast(DoubleType()))
# fill null values with default value
tweets = tweets.fillna(0.0, subset=["user_followers","user_friends","user_favourites"])
tweets = tweets.fillna(False, subset=["user_verified"])

root
-- user_name: string (nullable = true)
-- user_location: string (nullable = true)
-- user_description: string (nullable = true)
-- user_created: string (nullable = true)
-- user_followers: float (nullable = false)
-- user_friends: float (nullable = false)
-- user_favourites: float (nullable = false)
-- user_verified: boolean (nullable = false)
-- date: string (nullable = true)
-- text: string (nullable = true)
-- hashtags: string (nullable = true)
-- source: string (nullable = true)
-- positive_sentiment: double (nullable = true)
-- negative_sentiment: double (nullable = true)
-- is_retweet: string (nullable = true)

In [13]:


from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


# Read input columns location and hashtags - and annotate them as categorical values
indexLocation = StringIndexer(inputCol="user_location",outputCol="indexedLocations").setHandleInvalid("keep")
indexHashtages = StringIndexer(inputCol="hashtags",outputCol="indexedhashtags").setHandleInvalid("keep")

# assemble multiple columns into one feature column, we often use this with many Spark mllib out of the box machine learnig models
assembler = VectorAssembler(inputCols=[ "indexedLocations","indexedhashtags","user_followers", "user_friends","user_favourites","positive_sentiment","negative_sentiment","user_verified"], outputCol="features")

# StringIndexer: Read input column "is_retweet" (true/false) and annotate them as categorical values.
indexLabel = StringIndexer(inputCol="is_retweet", outputCol="indexedRetweetLabel").setHandleInvalid("keep")

# most often with decision tree we would want to normalize the data to make it fit into the bins:
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)

# DecisionTreeClassifier: Learn to predict column "indexedRetweetLabel" using the "normFeatures" column.
dtc = DecisionTreeClassifier(labelCol="indexedRetweetLabel",featuresCol="normFeatures")

# Chain indexer + dtc together into a single ML Pipeline.
pipeline = Pipeline(stages=[indexLocation,indexHashtages,assembler,indexLabel,normalizer, dtc])

# train our model
model = pipeline.fit(tweets)
# use our model - only for demo, for other cases we will use multiple datasets for testing and training
prediction = model.transform(tweets)


In [14]:
prediction.printSchema()

root
-- user_name: string (nullable = true)
-- user_location: string (nullable = true)
-- user_description: string (nullable = true)
-- user_created: string (nullable = true)
-- user_followers: float (nullable = false)
-- user_friends: float (nullable = false)
-- user_favourites: float (nullable = false)
-- user_verified: boolean (nullable = false)
-- date: string (nullable = true)
-- text: string (nullable = true)
-- hashtags: string (nullable = true)
-- source: string (nullable = true)
-- positive_sentiment: double (nullable = true)
-- negative_sentiment: double (nullable = true)
-- is_retweet: string (nullable = true)
-- indexedLocations: double (nullable = false)
-- indexedhashtags: double (nullable = false)
-- features: vector (nullable = true)
-- indexedRetweetLabel: double (nullable = false)
-- normFeatures: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [15]:
selected = prediction.select("is_retweet","user_location","hashtags","positive_sentiment","prediction")
for row in selected.collect():
    is_retweet,user_location, hashtags,positive_sentiment,prediction = row
    print("is_retweet=%s , user_location=%s , positive_sentiment=%s , hashtags=%s --> prediction=%f" % (str(is_retweet),str(user_location),str(positive_sentiment) ,str(hashtags), prediction))
    


is_retweet=false , user_location=Minnesota, USA , positive_sentiment=0.8046833395704176 , hashtags=['CyberRisk', 'COVID19'] --> prediction=0.000000
is_retweet=false , user_location=Thornhill-ish , positive_sentiment=0.853207351805683 , hashtags=['coronavirus'] --> prediction=0.000000
is_retweet=false , user_location=Worldwide (online & offline) , positive_sentiment=0.3256162941843388 , hashtags=['TheSource'] --> prediction=0.000000
is_retweet=false , user_location=Ottawa , positive_sentiment=0.7775005863033098 , hashtags=['COVID19'] --> prediction=0.000000
is_retweet=false , user_location=ACFAS, Chicago, IL , positive_sentiment=0.18529612986659694 , hashtags=['FootHealthFacts', 'COVID19'] --> prediction=0.000000
is_retweet=false , user_location=Washington, DC , positive_sentiment=0.6245762853112555 , hashtags=['SouthAsia', 'COVID19'] --> prediction=0.000000
is_retweet=false , user_location=Oregon Health & Science Univ , positive_sentiment=0.5561472758610895 , hashtags=['COVID19'] --> prediction=0.000000
is_retweet=false , user_location=Broken Britain 😢💔🇬🇧⚰️ , positive_sentiment=0.5963336549119539 , hashtags=['COVID19'] --> prediction=0.000000
is_retweet=false , user_location=Houston, Texas, USA , positive_sentiment=0.31841569830160155 , hashtags=['Houston', 'COVID19', 'Stupidity'] --> prediction=0.000000
is_retweet=false , user_location=Houston, TX , positive_sentiment=0.9176730870403939 , hashtags=['1A'] --> prediction=0.000000
is_retweet=false , user_location=Montclair, New Jersey USA , positive_sentiment=0.29510807137164075 , hashtags=['undocumented'] --> prediction=0.000000
is_retweet=false , user_location=Chicago, USA , positive_sentiment=0.0008954395443022678 , hashtags=['marketing', 'COVID19', 'DigitalMarketing', 'advertising'] --> prediction=0.000000
is_retweet=false , user_location=ÜT: 41.230596,-76.976382 , positive_sentiment=0.6515566125860556 , hashtags=['NCPA', 'COVID19'] --> prediction=0.000000
is_retweet=false , user_location=Texas , positive_sentiment=0.42520953760354563 , hashtags=['Medicaid'] --> prediction=0.000000
is_retweet=false , user_location=Manhattan, NY , positive_sentiment=0.55854183018708 , hashtags=['covid19hoax', 'coronavirushoax', 'covid19', 'coronavirus'] --> prediction=0.000000
is_retweet=false , user_location=Boca Raton, FL , positive_sentiment=0.5580706609283187 , hashtags=['GlobalPandemic', 'coronavirus', 'Republican'] --> prediction=0.000000
is_retweet=false , user_location=Islamabad, Pakistan , positive_sentiment=0.9014253094164878 , hashtags=['COVID19'] --> prediction=0.000000
is_retweet=false , user_location=India , positive_sentiment=0.7069508816784963 , hashtags=['COVID19'] --> prediction=0.000000
is_retweet=false , user_location=United States , positive_sentiment=0.8711439001127197 , hashtags=['MSM', 'COVID19'] --> prediction=0.000000
is_retweet=false , user_location=Colorado , positive_sentiment=0.9020311496348835 , hashtags=['COVID19', 'mobility'] --> prediction=0.000000
is_retweet=false , user_location=Canada , positive_sentiment=0.9240196905924654 , hashtags=['health', 'lifeinsurance', 'Covid19'] --> prediction=0.000000
is_retweet=false , user_location=Hoegaarden, Belgium , positive_sentiment=0.15933978815442507 , hashtags=['Covid19'] --> prediction=0.000000
is_retweet=false , user_location=Florida , positive_sentiment=0.18106922732629604 , hashtags=['Covid19'] --> prediction=0.000000
is_retweet=false , user_location=Mpls ➰ Tus , positive_sentiment=0.4270332513651981 , hashtags=['COVID19'] --> prediction=0.000000
is_retweet=false , user_location=India , positive_sentiment=0.17529118382522502 , hashtags=['COVID19'] --> prediction=0.000000
is_retweet=false , user_location=The Milky Way , positive_sentiment=0.0517550824774583 , hashtags=['Narcissism', 'COVID19', 'coviddeniers', 'Massachusetts'] --> prediction=0.000000
is_retweet=false , user_location=New York, USA , positive_sentiment=0.45568398962169687 , hashtags=['Form941', 'COVID19'] --> prediction=0.000000
is_retweet=false , us

In [16]:
# persist the model to disk
model.save("/mnt/root/COVID19_TWEETS/ML-Models/V1")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1466433438281981> in <module> 
 1 # persist the model to disk 
 ----> 2 model . save ( "/mnt/root/COVID19_TWEETS/ML-Models/V1" ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 238 def save ( self , path ) : 
 239 """Save this ML instance to the given path, a shortcut of 'write().save(path)'.""" 
 --> 240 self . write ( ) . save ( path ) 
 241 
 242 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 189 if not isinstance ( path , basestring ) : 
 190 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 191 self . _jwrite . save ( path ) 
 192 
 193 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 126 def deco ( * a , ** kw ) : 
 127 try : 
 --> 128 return f ( * a , ** kw ) 
 129 except py4j . protocol . Py4JJavaError as e : 
 130 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1899.save.
: java.io.IOException: Path /mnt/root/COVID19_TWEETS/ML-Models/V1 already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.super$save(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$4(Pipeline.scala:344)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(events.scala:177)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent$(events.scala:172)
	at org.apache.spark.ml.util.Instrumentation.withSaveInstanceEvent(Instrumentation.scala:43)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3$adapted(Pipeline.scala:344)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:284)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:284)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.save(Pipeline.scala:344)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)